In [12]:
import pandas as pd

In [21]:
url = "./make_file/(위_경도)_서울시_도로_정리(기점_종점).csv"

In [22]:
data = pd.read_csv(url, encoding = "UTF-8")

In [23]:
data.columns

Index(['시점_위도', '시점_경도', '종점_위도', '종점_경도', '길이'], dtype='object')

In [24]:
try :
    data = data.drop(columns='Unnamed: 0')
except KeyError as e :
    print(e)

"['Unnamed: 0'] not found in axis"


In [25]:
print(len(data))

569


In [26]:
data.columns

Index(['시점_위도', '시점_경도', '종점_위도', '종점_경도', '길이'], dtype='object')

In [27]:
print(data.isnull().sum())

시점_위도    0
시점_경도    0
종점_위도    0
종점_경도    0
길이       0
dtype: int64


In [28]:
start_latitude = data['시점_위도'].to_list()
start_longitude = data['시점_경도'].to_list()

end_latitude = data['종점_위도'].to_list()
end_longitude = data['종점_경도'].to_list()

start_altitude = []
end_altitude = []

In [29]:
print(len(start_latitude))
print(len(start_longitude))
print(len(end_latitude))
print(len(end_longitude))

569
569
569
569


## GOOGLE API

In [30]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
open_api_key_file['사이트'].value_counts()


사이트
공공데이터                 1
kakao_api_key_REST    1
카카오 JS                1
국토부v-world            1
GOOGLE_MAP            1
행정안전부                 1
Name: count, dtype: int64

In [31]:
GOOGLE_API_KEY = open_api_key_file[open_api_key_file['사이트'] == 'GOOGLE_MAP'].values[0][1]   

In [32]:
import requests
import time
import pandas as pd 

def google_api(latitudes, longitudes, mode):
    global start_altitude, end_altitude
    
    url = f"https://maps.googleapis.com/maps/api/elevation/json?key={GOOGLE_API_KEY}"

    locations = list(zip(latitudes, longitudes))  # (위도, 경도) 쌍 리스트
    
    for i in range(0, len(locations), 400):  # 최대 512개씩 요청
        batch_locations = locations[i:i+400]  # 최대 512개씩 묶기
        locations_param = '|'.join([f"{lat},{lon}" for lat, lon in batch_locations])
        full_url = f"{url}&locations={locations_param}"
        
        response = requests.get(full_url)
            
        if response.status_code == 200:
            data = response.json()
            if data['status'] == 'OK':
                elevations = [result['elevation'] for result in data['results']]   
            
                if mode == 'start':
                    start_altitude.extend(elevations)
                elif mode == 'end':
                    end_altitude.extend(elevations)
            else:
                print("Error in response:", data['status'])
        else:
            print("API 호출 실패:", response.status_code)

        time.sleep(0.1)


In [33]:
print(f'전) 기점 고도 길이 : {len(start_altitude)}') 

print(f'전) 종점 고도 길이 : {len(end_altitude)}') 

전) 기점 고도 길이 : 0
전) 종점 고도 길이 : 0


In [34]:
# API 호출 (start)
print(f'기점의 고도값 불러오기')
print(f'시작') 
google_api(start_latitude, start_longitude, 'start') 
print(f'종료')

기점의 고도값 불러오기
시작
종료


In [35]:
# API 호출 (end)
print(f'종점의 고도값 불러오기')
print(f'시작')
google_api(end_latitude, end_longitude, 'end')
print(f'종료')

종점의 고도값 불러오기
시작
종료


In [36]:
print(f'후) 기점 고도 길이 : {len(start_altitude)}') 
print(f'후) 종점 고도 길이 : {len(end_altitude)}') 

if (len(start_altitude) == len(end_altitude)) and (len(start_altitude) > 0) :
    print(f'결과의 길이가 같습니다.')
    print(f'성공적으로 처리되었습니다.')
else :
    print(f'데이터 처리에 실패했습니다...')

후) 기점 고도 길이 : 569
후) 종점 고도 길이 : 569
결과의 길이가 같습니다.
성공적으로 처리되었습니다.


In [37]:
altitude_difference = []

for i in range(len(start_altitude)) :
    result = abs(start_altitude[i] - end_altitude[i]) 
    altitude_difference.append(result)

print(len(altitude_difference))

569


In [38]:
# 결과 데이터 추가 및 저장
data['시작점_고도'] = start_altitude
data['종료점_고도'] = end_altitude
data['고도의_차이'] = altitude_difference

In [39]:
data.columns

Index(['시점_위도', '시점_경도', '종점_위도', '종점_경도', '길이', '시작점_고도', '종료점_고도', '고도의_차이'], dtype='object')

In [40]:
start_lat = data['시점_위도'].to_list()
start_lon = data['시점_경도'].to_list()
end_lat = data['종점_위도'].to_list()
end_lon = data['종점_경도'].to_list()
different_alt = data['고도의_차이'].to_list()

In [41]:
print(f'start_lat 길이 : {len(start_lat)}')
print(f'start_lon 길이 : {len(start_lon)}')
print(f'end_lat 길이 : {len(end_lat)}')
print(f'end_lon 길이 : {len(end_lon)}')
print(f'different_alt 길이 : {len(different_alt)}')

start_lat 길이 : 569
start_lon 길이 : 569
end_lat 길이 : 569
end_lon 길이 : 569
different_alt 길이 : 569


In [42]:
import math

# 지구 반지름 (미터)
R = 6371000   

In [43]:
# 결과 저장 리스트
road_slope = []

In [44]:
for i in range(len(start_lat)):
    # 위경도를 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(math.radians, [start_lat[i], start_lon[i], end_lat[i], end_lon[i]])

    # 위도, 경도의 차이
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine 공식으로 수평 거리(d) 계산
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c  # 두 지점 간 수평 거리 (m)

    # 고도 차이 (h)
    h = different_alt[i]  

    # 경사각(θ) 계산 (라디안 → 도 변환)
    theta = math.degrees(math.atan2(h, d))

    # 결과 저장
    road_slope.append(theta)

    if(i < 5) :
        # 디버깅 출력
        print(f"Point {i + 1}: d = {d:.4f} m, h = {h:.4f} m, θ = {theta:.4f}°")


Point 1: d = 153.0020 m, h = 31.7527 m, θ = 11.7242°
Point 2: d = 434.7030 m, h = 38.3723 m, θ = 5.0446°
Point 3: d = 289.7552 m, h = 39.9972 m, θ = 7.8593°
Point 4: d = 198.7964 m, h = 35.2075 m, θ = 10.0431°
Point 5: d = 307.6295 m, h = 6.5866 m, θ = 1.2266°


In [45]:
print(f'변환한 road_slope의 길이 : {len(road_slope)}')

변환한 road_slope의 길이 : 569


In [46]:
new_add_df = pd.DataFrame({
    '경사각' : road_slope
})

In [47]:
data['경사각'] = new_add_df

In [48]:
data.columns

Index(['시점_위도', '시점_경도', '종점_위도', '종점_경도', '길이', '시작점_고도', '종료점_고도', '고도의_차이',
       '경사각'],
      dtype='object')

In [49]:
data['경사각']

0      11.724227
1       5.044559
2       7.859327
3      10.043141
4       1.226563
         ...    
564     0.264654
565     0.348516
566     0.521437
567     0.572816
568     0.740789
Name: 경사각, Length: 569, dtype: float64

In [50]:
# 중점 좌표 계산
mid_lat = [(start + end) / 2 for start, end in zip(start_lat, end_lat)]
mid_lon = [(start + end) / 2 for start, end in zip(start_lon, end_lon)]

# 데이터프레임에 중점 좌표 추가
data['중점_위도'] = mid_lat
data['중점_경도'] = mid_lon

In [51]:
data.columns

Index(['시점_위도', '시점_경도', '종점_위도', '종점_경도', '길이', '시작점_고도', '종료점_고도', '고도의_차이',
       '경사각', '중점_위도', '중점_경도'],
      dtype='object')

In [52]:
data

,시점_위도,시점_경도,종점_위도,종점_경도,길이,시작점_고도,종료점_고도,고도의_차이,경사각,중점_위도,중점_경도
0,37.589926,126.993697,37.591160,126.992929,160,62.080006,93.832680,31.752674,11.724227,37.590543,126.993313
1,37.611574,126.978609,37.615247,126.976917,150,149.520370,187.892639,38.372269,5.044559,37.613411,126.977763
2,37.610064,126.975395,37.612667,126.975238,210,118.210579,158.207809,39.997231,7.859327,37.611366,126.975317
3,37.606240,126.957833,37.604560,126.957061,190,92.292526,127.500053,35.207527,10.043141,37.605400,126.957447
4,37.583442,126.985409,37.583186,126.988885,355,48.981506,42.394905,6.586601,1.226563,37.583314,126.987147
...,...,...,...,...,...,...,...,...,...,...,...
564,37.552091,127.143855,37.552047,127.142747,110,32.475735,32.927330,0.451595,0.264654,37.552069,127.143301
565,37.551713,127.143822,37.551680,127.142719,110,32.412693,33.004562,0.591869,0.348516,37.551697,127.143270
566,37.551245,127.143771,37.551306,127.142668,110,32.272240,33.159863,0.887623,0.521437,37.551275,127.143220
567,37.540927,127.149979,37.541814,127.149292,90,30.534058,29.376165,1.157892,0.572816,37.541371,127.149635


In [53]:
try :
    save_data_url = './make_file/열선_위경도값(중앙값_열선_경사각_포함).csv'
    data.to_csv(save_data_url, encoding='euc-kr')
    print(f"{save_data_url} 저장 완료")
except :
    print("err") 

./make_file/열선_위경도값(중앙값_열선_경사각_포함).csv 저장 완료
